# Ranking Strategies Notebook
This notebook normalized cumulative, reliability, and decay metrics and ranks strategies highest to lowest based on these metrics.

## Import Libraries

In [1]:
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import  util
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from EquityHedging.analytics import hedge_metrics as hm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
    


## Import returns data

### Imports Daily, Weekly, Monthly, Quarterly and Yearly returns data

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**) - SPTR, M1WD, SX5T
* Decide if you want a Fixed Income Benchmark (**include_fi**) - True, False
* Select what equity hedging strategies to exclude (**strat_drop_list**). Below is the list of strategies:

    * 99%/90% Put Spread, Down Var, Vortex, VOLA , Dynamic Put Spread, VRR, GW Dispersion, Corr Hedge

In [25]:
equity_bmk = 'M1WD'
include_fi = False
weighted = [True, False]
strat_drop_list = ['99%/90% Put Spread', 'Vortex']
new_strat = True
returns = dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

##  Get Notional Weights 

In [3]:
#notional_weights = dm.get_notional_weights(returns['Monthly'])
notional_weights = [19.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.25, 1.0]
df_weights = get_df_weights(notional_weights, list(returns['Monthly'].columns), include_fi)
fmt.get_notional_styler(df_weights)

,M1WD,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var
Notional Weights (Billions),$19.00,$1.00,$1.00,$1.00,$1.00,$1.00,$0.25,$1.00
Percentage Weights,100.00%,5.26%,5.26%,5.26%,5.26%,5.26%,1.32%,5.26%
Strategy Weights,0.00%,16.00%,16.00%,16.00%,16.00%,16.00%,4.00%,16.00%


In [26]:
temp_ret = returns['Weekly'].copy()

In [29]:
returns['Weekly'] = temp_ret.copy()
returns['Weekly']['VOLA 3'] = returns['Weekly']['VOLA'] *.3
returns['Weekly']['Def Var 3'] = returns['Weekly']['Def Var'] *.3
returns['Weekly'] = returns['Weekly'][['M1WD','Down Var', 'VOLA', 'Dynamic Put Spread', 'VRR', 'GW Dispersion',
       'Corr Hedge', 'Def Var','VOLA 3', 'Def Var 3']]
notional_weights = [19.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.25, 1.0,1.0,1.0]

In [30]:
df_sample = hm.get_hedge_metrics_to_normalize(returns, equity_bmk, notional_weights, weighted_hedge = True)
#format hedge metrics
#show hedge metrics

In [31]:
df_sample

,Benefit,Downside Reliability,Upside Reliability,Convexity,Cost,Decay
Down Var,1.384568,-0.378368,0.245336,0.950600,-2.029530,210.0
VOLA,0.630722,-0.759925,-0.383986,1.246549,-1.661032,322.0
Dynamic Put Spread,0.892500,-0.649971,-0.602008,0.259934,-1.260329,200.0
VRR,2.316304,-0.066891,-0.009208,0.541243,-2.178475,133.0
GW Dispersion,1.040725,-0.474068,0.030755,0.345939,-1.032908,361.0
Corr Hedge,1.261181,-0.138395,0.124009,1.087390,-1.202728,333.0
Def Var,1.343204,-0.047311,-0.077938,0.223392,-0.465385,308.0
VOLA 3,0.189217,-0.759925,-0.383986,0.373965,-0.498310,365.0
Def Var 3,0.402961,-0.047311,-0.077938,0.067018,-0.139615,365.0
Weighted Hedges,0.527774,-0.687047,-0.157365,0.356365,-0.492658,365.0


In [32]:
x = util.reverse_signs_in_col(df_sample,'Downside Reliability')
x

,Benefit,Downside Reliability,Upside Reliability,Convexity,Cost,Decay
Down Var,1.384568,0.378368,0.245336,0.950600,-2.029530,210.0
VOLA,0.630722,0.759925,-0.383986,1.246549,-1.661032,322.0
Dynamic Put Spread,0.892500,0.649971,-0.602008,0.259934,-1.260329,200.0
VRR,2.316304,0.066891,-0.009208,0.541243,-2.178475,133.0
GW Dispersion,1.040725,0.474068,0.030755,0.345939,-1.032908,361.0
Corr Hedge,1.261181,0.138395,0.124009,1.087390,-1.202728,333.0
Def Var,1.343204,0.047311,-0.077938,0.223392,-0.465385,308.0
VOLA 3,0.189217,0.759925,-0.383986,0.373965,-0.498310,365.0
Def Var 3,0.402961,0.047311,-0.077938,0.067018,-0.139615,365.0
Weighted Hedges,0.527774,0.687047,-0.157365,0.356365,-0.492658,365.0


In [33]:
df_normal = util.get_normalized_data(x)
df_normal.transpose()

,Benefit,Downside Reliability,Upside Reliability,Convexity,Cost,Decay
Down Var,0.561966,4.645676e-01,1.000000,0.749097,0.073053,0.331897
VOLA,0.207563,1.000000e+00,0.257301,1.000000,0.253790,0.814655
Dynamic Put Spread,0.330632,8.457031e-01,0.000000,0.163553,0.450323,0.288793
VRR,1.000000,2.747651e-02,0.699599,0.402045,0.000000,0.000000
GW Dispersion,0.400317,5.988622e-01,0.746761,0.236468,0.561867,0.982759
Corr Hedge,0.503959,1.278169e-01,0.856815,0.865066,0.478575,0.862069
Def Var,0.542520,0.000000e+00,0.618485,0.132574,0.840220,0.754310
VOLA 3,0.000000,1.000000e+00,0.257301,0.260228,0.824071,1.000000
Def Var 3,0.100487,3.053113e-16,0.618485,0.000000,1.000000,1.000000
Weighted Hedges,0.159165,8.977311e-01,0.524749,0.245307,0.826843,1.000000


## Get Normalized Data

In [4]:
df_normal= summary.get_normalized_hedge_metrics(returns, equity_bmk, notional_weights, weighted_hedge = True)
df_normal
#TODO: format the data to only have 2 decimal

,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var,Weighted Hedges
Benefit,0.447231,0.000000,0.155304,1.000000,0.243241,0.374030,0.422692,0.007034
Downside Reliability,0.464568,1.000000,0.845703,0.027477,0.598862,0.127817,0.000000,0.888764
Upside Reliability,1.000000,0.257301,0.000000,0.699599,0.746761,0.856815,0.618485,0.567331
Convexity,0.710750,1.000000,0.035714,0.310656,0.119773,0.844444,0.000000,0.186045
Cost,0.086945,0.302052,0.535959,0.000000,0.668714,0.569583,1.000000,0.919781
Decay,0.337719,0.828947,0.293860,0.000000,1.000000,0.877193,0.767544,0.951754


In [5]:
df_normal.transpose()

,Benefit,Downside Reliability,Upside Reliability,Convexity,Cost,Decay
Down Var,0.447231,0.464568,1.000000,0.710750,0.086945,0.337719
VOLA,0.000000,1.000000,0.257301,1.000000,0.302052,0.828947
Dynamic Put Spread,0.155304,0.845703,0.000000,0.035714,0.535959,0.293860
VRR,1.000000,0.027477,0.699599,0.310656,0.000000,0.000000
GW Dispersion,0.243241,0.598862,0.746761,0.119773,0.668714,1.000000
Corr Hedge,0.374030,0.127817,0.856815,0.844444,0.569583,0.877193
Def Var,0.422692,0.000000,0.618485,0.000000,1.000000,0.767544
Weighted Hedges,0.007034,0.888764,0.567331,0.186045,0.919781,0.951754


In [12]:
df_normal=d.copy()
df_normal.drop(['Weighted Hedges'], axis=1, inplace=True)


In [13]:
df_normal.columns

Index(['Down Var', 'VOLA', 'Dynamic Put Spread', 'VRR', 'GW Dispersion',
       'Corr Hedge', 'Def Var', 'Weighted Hedges', 'vola 3', 'def var 3'],
      dtype='object')

## Creates a List of Symbols 

In [34]:
#get length of columns
n=len(df_normal.columns)

#get column names of df_normal
c=list(df_normal.columns)

#creates a list of the corresponding number of the symbols we want
a=list(range(0,33))
b=list(range(46,53))
for i in b:
    a.append(i)
    
#Creates a data frame that assigns each strategy with a symbol
symbols=pd.DataFrame(columns=c,index=[0])
symbols.iloc[0] = a[0:n]

#create new method in util
#make weighted hedges fixed symbol


## Graph Normalized Data and Ranking Each Strategy

In [35]:
#transpose normal data here instead of in method
index = list(df_normal.index)
fig = go.Figure()
#loops through eaach column and plots each metric for the given strategy
for col in df_normal.columns:
    fig.add_trace(go.Scatter(
        x=index,
        y=df_normal[col],
        marker=dict(size = 20),
        marker_symbol = symbols[col],
        mode= "markers",
        name=col
        ))
fig.update_layout(title="",
                xaxis_title="Hedge Metric",
                yaxis_title="Higher the Better")
fig.show()